In [3]:
!pip install opencv-python

/bin/bash: /home/u/anaconda3/envs/dxtf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

transform = transforms.Compose(
    [
        transforms.Grayscale(),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
    ]
)

train_data = datasets.ImageFolder("dataset/train", transform=transform)
test_data = datasets.ImageFolder("dataset/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

for epoch in range(150):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_loss = running_loss / len(train_loader.dataset)
    accuracy = 100.0 * correct / total
    print(f"ResNet Epoch {epoch+1}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.2f}%")

torch.save(model.state_dict(), "resnet_handwritten.pth")
import json

json.dump(train_data.classes, open("classes.json", "w"))
print("Model and class names saved.")

ResNet Epoch 1: Loss = 1.4583, Accuracy = 64.90%
ResNet Epoch 2: Loss = 0.8014, Accuracy = 79.44%
ResNet Epoch 3: Loss = 0.6530, Accuracy = 83.36%
ResNet Epoch 4: Loss = 0.5732, Accuracy = 85.34%
ResNet Epoch 5: Loss = 0.5189, Accuracy = 86.84%
ResNet Epoch 6: Loss = 0.4837, Accuracy = 87.75%
ResNet Epoch 7: Loss = 0.4583, Accuracy = 88.38%
ResNet Epoch 8: Loss = 0.4370, Accuracy = 88.98%
ResNet Epoch 9: Loss = 0.4181, Accuracy = 89.55%
ResNet Epoch 10: Loss = 0.4004, Accuracy = 90.08%
ResNet Epoch 11: Loss = 0.3985, Accuracy = 90.11%
ResNet Epoch 12: Loss = 0.3827, Accuracy = 90.61%
ResNet Epoch 13: Loss = 0.3766, Accuracy = 90.84%
ResNet Epoch 14: Loss = 0.3618, Accuracy = 91.25%
ResNet Epoch 15: Loss = 0.3647, Accuracy = 91.22%
ResNet Epoch 16: Loss = 0.3546, Accuracy = 91.49%
ResNet Epoch 17: Loss = 0.3512, Accuracy = 91.69%
ResNet Epoch 18: Loss = 0.3472, Accuracy = 91.80%
ResNet Epoch 19: Loss = 0.3418, Accuracy = 91.90%
ResNet Epoch 20: Loss = 0.3386, Accuracy = 92.03%
ResNet Ep